Este projeto você irá implementar o processamento de consultas. Nela, você utilizará o índice para retornar uma coleção ordenada e avaliação de algumas consultas selecionadas. Para isso, vocês deverão implementar alguns métodos das seguintes classes.

- `IndexPreComputedVals`: Em alguns modelos, há a necessidade de processar alguns valores para que, no momento da execução da consulta, seja retornado de forma mais rápida. Esta classe analisa o índice e armazena informações necessárias para o calculo de cada tipo de modelagem;
- `RankingModel`: Classe abstrata para a criação dos modelos. Ele possui o método `get_ordered_docs` a ser implementado por suas subclasses;
- `BooleanRankingModel` Classe que retorna um resultado de consulta por meio do [modelo booleano](https://docs.google.com/presentation/d/1V62ll_IXRrsp6TYUHjx_T4jIyIc1ZVJYoOSwsxObybE/edit?usp=sharing)
- `VectorRankingModel`: Classe que retorna um resultado de consulta por meio do [modelo vetorial](https://docs.google.com/presentation/d/1jsD1MpLIl08OnWysDhjp7glc4_K0sH9sKUhRqI8lLLo/edit?usp=sharing)
- `QueryRunner`: Classe principal encarregada de obter a consulta e retornar os resultados;


Este trabalho depende do código do indice (pacote `index`). Assim, você deve adicioná-o apropriadamente para dar continuidade ao projeto. 


## Modelagem Booleana e Vetorial

**Atividade 1 - Modelagem Booleana**: A abordagem booleana neste trabalho é simplificada. O modelo recebe o atributo `operator` que é uma instancia do [Enum](https://docs.python.org/3.4/library/enum.html) Operator. Caso o operador seja AND, será feito a operacao de interseção entre todos os documentos contidos ocorrencias de palavras, caso contrario, sendo OR, será feito a união. Exemplo: 

In [1]:
from index.index.structure import TermOccurrence
map_ocorrencias = {"saturno":[TermOccurrence(1,1,1),
                            TermOccurrence(3,1,1)],
                     "plutao":[TermOccurrence(2,5,1),
                               TermOccurrence(4,5,1)],
                        "terra":[TermOccurrence(1,2,1),
                            TermOccurrence(2,2,1),
                            TermOccurrence(4,2,1),],
                        "venus":[TermOccurrence(1,3,1),
                                TermOccurrence(2,3,1),
                                TermOccurrence(3,3,1),
                                TermOccurrence(4,3,1)],
                        "marte":[TermOccurrence(1,4,2),
                            TermOccurrence(3,4,1),
                            TermOccurrence(4,4,1),],

                        "mercurio":[TermOccurrence(3,6,1)]          
    }

In [2]:
{1,2,3}|{2,4}

{1, 2, 3, 4}

A intereseção entre `saturno` e `venus` resultará nos documentos 1 e 3 e, a união, nos documentos 1, 2, 3 e 4.

Esta é uma forma bem simplificada para implementarmos o modelo booleano. Para isso, você deverá implementar os métodos `union_all` e `intersection_all` presentes na classe `BooleanRankingModel` no arquivo `ranking_models.py`. Esses métodos recebem como parâmetro um mapa com a lista de correncias de cada termo (similar a exemplificada acima).

In [3]:
!python -m query.tests.ranking_models RankingModelTest.test_boolean_model

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


**Atividade 2 - TF-IDF:** Agora, no mesmo arquivo, iremos finalizar a implementação do modelo Vetorial utilizando a classe `VectorRankingModel`. Nesta classe, primeiramente, você deverá implementar os [métodos estáticos](https://daniel-hasan.github.io/cefet-web-grad/classes/python2/) `tf`, `idf` e `tf_idf`. Sendo que $TF = 1+log_2(f_{ij})$ e $IDF_i = log_2(\frac{N}{n_i})$ em que $f_{ij}$ é a frequência do termo $i$ no documento $j$, $N$ é o número de documentos e $n_i$ é o número de documentos que ocorrem o termo $i$. Abaixo, faça testes destes métodos: 

In [4]:
from query.ranking_models import VectorRankingModel

doc_count = 300
term_frequency = 60
doc_frequency = 30

tf = VectorRankingModel.tf(term_frequency)
idf = VectorRankingModel.idf(doc_count, doc_frequency)
tf_idf = VectorRankingModel.tf_idf(doc_count, term_frequency, doc_frequency)

print(f"tf: {tf}")
print(f"idf: {idf}")
print(f"tf_idf: {tf_idf}")

tf: 6.906890595608519
idf: 3.3219280948873626
tf_idf: 22.94419391786525


**Atividade 3 - PreComputedVals:** No modelo vetorial temos que calcular a norma de cada documento $d_j$. Esse cálculo pode ser feito durante o preprocessamento da consulta. Assim, a classe `IndexPreComputedVals` possui o atributo `document_norm` que é um dicionário que mapeia cada documento $j$ à sua norma. Esse calculo é feito apenas uma vez ao iniciar o programa. 

Desta forma, você deverá terminar de implementar o método `precompute_vals` que percorre todo o índice e armazena a norma de cada documento.

In [5]:
!python -m query.tests.ranking_models RankingModelTest.test_precomputed_vals

.
----------------------------------------------------------------------
Ran 1 test in 0.095s

OK


**Atividade 4 - Método `get_ordered_docs` da classe `VectorRankingModel`:** Usando os métodos implementados anteriormente você deverá ordenar os documentos contidos no mapa de ocorrencias `docs_occur_per_term` de acordo com a consulta `query` utilizando o modelo vetorial. O parametro `query` mapeia um termo presente na consulta, para a sua ocorrencia (objeto da classe `TermOcurrence`) na propria consulta. 

Para cada termo $t$ que ocorre na consulta, `docs_occur_per_term` mapeia cada termo com a lista de ocorrencias dele no índice. Veja abaixo um exemplo destes parametros usando a consulta `to be or not to be`.  Veja que, na consulta, `doc_id = None`.

In [6]:
from index.index.structure import TermOccurrence
query = {
    "to":TermOccurrence(None, 1, 2),
    "be":TermOccurrence(None, 2, 2),
    "or":TermOccurrence(None, 3, 1),
    "not":TermOccurrence(None, 4, 1),
}

docs_occur_per_term = {
    "to":[TermOccurrence(1, 1, 4), TermOccurrence(2, 1, 1),],
    "be":[TermOccurrence(1, 2, 1),TermOccurrence(2, 2, 1)],
    "or":[TermOccurrence(2, 3, 1)],
    "not":[TermOccurrence(3, 4, 1)],
}

Nesse exemplo, temos a consulta (representado pela variavel `query`) 'to to be be or not', ou seja, o `to` e o `be` ocorrendo duas vezes na consulta e, os demais termos, uma vez - a ordem não é definida no parametro. Em `docs_occcur_per_term` temos a ocorrencia desses termos nos documentos da coleção. 

Você deve executar o modelo vetorial para obter o resultado `documents_weight` que mapeia, para cada documento a similaridade entre ele e a consulta utilizando o modelo vetorial e a distancia do cosseno. Note que neste método você **não** pode navegar por todos os documentos da coleção pois, caso seja feito isso, o código de vocês iriam demorar muito caso sua coleção tiver milhões ou bilhões de documentos. Uma dica é usar o `documents_weight` para armazenar os valores intermediarios do somatorio de $w_{ij} \times  w_{iq}$, tais variáveis são definidas nos [slides de modelagem vetorial](https://docs.google.com/presentation/d/1jsD1MpLIl08OnWysDhjp7glc4_K0sH9sKUhRqI8lLLo/edit?usp=sharing). 



Esse método retorna dois valores: (a) uma lista de ids de documentos ordenada de acordo com o modelo vetorial - use o método e um dicionário que mapeia, para cada documento, o seu peso. O método `rank_document_ids` será útil. 

In [3]:
!python -m query.tests.ranking_models RankingModelTest.test_vector_model

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


## Processamento da Consulta e Avaliação

Agora você irá fazer o processamento da consulta, informada pelo usuário, além de sua avaliação. A implementação do processamento de consultas será feito na classe `QueryRunner` do arquivo `processing.py`.

**Requisito antes de começar:** o código que foi feito da indexação deve estar funcionando. Será utilizado a base de dados da Wikipédia. Você não deverá fazer a indexação toda quando iniciar o programa, ao invés disso, você deve persistir o indice todo em arquivo após a indexação. Usando FileIndex, como as ocorrencias já estão armazenadas em arquivo, você precisa armazenar apenas o conteúdo do `dic_index`. A [biblioteca json](https://docs.python.org/3/library/json.html) pode ajudar. Este índice será lido do arquivo apenas uma vez no início da execução do programa.

**Criação da coleção de referência:** Para o projeto realizaremos uma avaliação bem simples, com o único intuito de simularmos um processo real de avaliação. Para tanto, consideraremos como conjunto de consultas de teste apenas três consultas:
'Irlanda'
'Belo Horizonte' e 
'São Paulo'

O conjunto de documentos de teste compreenderá todas as páginas da base de dados da Wikipédia PT-BR utilizadas no projeto.  Para cada consulta, disponibilizamos um arquivo (na pasta `relevant_docs`) com o id de documentos relevantes (separados por vírgula) para as consultas teste.

Por exemplo, um documento $D$ será considerado relevante para a consulta 'Belo Horizonte' somente 
se o id de $D$ estiver no arquivo `belo_horizonte.dat`. Você irá armazenar o conteúdo desses arquivos em memória para diminuir o tempo de busca. Feito isso, a coleção de referência para as três consultas estará montada e pode-se realizar os cálculos de avaliação corretamente.

**Como um artigo foi considerado relevante para uma determinada consulta?** A Wikipedia organiza seus artigos em diversas categorias. Assim, para considerarmos se um artigo da Wikipédia é relevante, utilizamos essas categorias. Assim, para os documentos relevantes para a consulta 'Irlanda' (Arquivo `irlanda.dat`), foram considerados relevantes artigos da seguintes categorias: 
 
- Irlanda
- Economia da Irlanda
- História da Irlanda
- Cultura da Irlanda
- Romancistas da Irlanda
- Físicos da Irlanda
- Reis da Irlanda
- Lordes da Irlanda

Categorias relevantes para a consulta 'Belo Horizonte' (Arquivo `belo_horizonte.dat`): 

- Bairros de Belo Horizonte
- Bandas de Belo Horizonte
- Belo Horizonte
- Edifícios de Belo Horizonte
- Metrô de Belo Horizonte
- Naturais de Belo Horizonte
- Prefeitos de Belo Horizonte
- Vereadores de Belo Horizonte

Categorias relevantes para a consulta 'São Paulo' (arquivo `sao_paulo.dat`):

- Atrações turísticas da cidade de São Paulo
- Áreas protegidas de São Paulo
- Prefeitos de São Paulo
- São Paulo
- Turismo em São Paulo
- Universidades de São Paulo
- Rodovias de São Paulo
- Museus da cidade de São Paulo
- Governadores de São Paulo
- Municípios de São Paulo

**Atividade 5 - Leitura dos arquivos de relevâncias:** Você deverá implementar o método `get_relevance_per_query` que irá ler todos os arquivo da pasta `relevant_docs` e, com isso, retornar um mapeamento em que a chave é a string de consulta e o valor é o **conjunto** de ids de documentos. Veja um exemplo de retorno com as consultas `Bolívia`, `Brasil` e `Porto Seguro`: 


In [8]:
retorno = {"Bolívia":{1,3,5,6,233},
            "Brasil":{2,4,5,3},
           "Porto Seguro":{3,43,21,3,12,233}
          }

Faça um teste de execução abaixo: 

In [9]:
!python -m query.tests.processing ProcessingTest.test_get_relevance_per_query

{'belo_horizonte': {'107302', '69168', '101043', '29241', '19588', '97000', '47967', '125549', '108604', '43658', '80198', '91853', '484', '110030', '104137', '136473\n', '18111', '35985', '136473', '130173', '1083', '40226', '49186', '91593', '38636', '106575', '49772'}, 'irlanda': {'46441', '52998', '102888', '1393', '83228', '33833', '58189', '51778', '3765', '51779', '52995', '15393', '105145', '51714', '1034', '53004', '95773', '111966', '39300', '37935', '1196', '37632', '11953', '93223', '44259', '13256', '51786', '51716', '105348', '7765', '26276', '39049', '47185', '124818\n', '9955', '43872', '9918', '54836', '45577'}, 'sao_paulo': {'37242', '64679', '34812', '34932', '41967', '34786', '34800', '35058', '120719', '35183', '18305', '35074', '35177', '34832', '59093', '64678', '16722', '35076', '61190', '63827', '113733', '34808', '34936', '37249', '34898', '50674', '64326', '54087', '57810', '63584', '110315', '72497', '125623', '32353', '35066', '23988', '35093', '35155', '40

.
----------------------------------------------------------------------
Ran 1 test in 0.091s

OK


**Atividade 6 - count_top_n_relevant:** Um método que auxilirará vocês na avaliação é o método count_top_n_relevant da classe Query Runner.  Esse método calcula a quantidade de documentos relevantes nas top `n` posições da lista `lstResposta` que é a resposta a uma consulta - lista de ids de documentos. `lstResposta` será a lista de respostas ordenadas por um método de processamento de consulta (BM25, Modelo vetorial). Os ids de documentos relevantes estão no parametro `docRelevantes`.

In [10]:
!python -m query.tests.processing ProcessingTest.test_count_top_n_relevant

.
----------------------------------------------------------------------
Ran 1 test in 0.094s

OK


**Atividade 7 - processamento da consulta**: O método `get_query_term_occurence` a consulta da mesma forma que foi preprocessado o texto do documento (use a classe `Cleaner` para isso). Este método irá retonar a consulta em um dicionario em que chave é o termo que ocorreu e o valor é uma instancia da classe TermOccurrence (ver Atividade 4). O doc_id deverá ser sempre None. Caso o termo nao exista no indice, ele será desconsiderado.

In [11]:
!python -m query.tests.processing ProcessingTest.test_get_query_term_occurence

Consulta: crocodilo
Resposta do método: {}

Consulta: vocês
Resposta do método: {'vocês': ( doc: None term_id:1 freq: 1)}

Consulta: Vocês estejam
Resposta do método: {'vocês': ( doc: None term_id:1 freq: 1), 'estejam': ( doc: None term_id:4 freq: 1)}

Consulta: vocês vocês crocodilo
Resposta do método: {'vocês': ( doc: None term_id:1 freq: 2)}



.
----------------------------------------------------------------------
Ran 1 test in 0.101s

OK


**Atividade 8 - Recuperação dos termos da consulta no índice:** O método `get_occurrence_list_per_term` possui com parametro a lista de termos da consulta. Este método retorna um dicionario com a lista de ocorrencia no indice de cada termo passado como parametro. Caso o termo não exista, este termo possuirá uma lista vazia. Veja o exemplo na atividade 4.

In [12]:
!python -m query.tests.processing ProcessingTest.test_get_occurence_list_per_term

Termos: ['crocodilo']
Resposta do método: {'crocodilo': []}

Termos: ['vocês', 'estejam', 'crocodilo']
Resposta do método: {'vocês': [( doc: 2 term_id:1 freq: 3), ( doc: 3 term_id:1 freq: 1)], 'estejam': [( doc: 3 term_id:4 freq: 1)], 'crocodilo': []}



.
----------------------------------------------------------------------
Ran 1 test in 0.096s

OK


**Atividade 9 - processamento da consulta:** Este método recebe como parametro a consulta, os valores precomputado do índice e o dicionário de documentos relevantes (extraídos do método `get_relevance_per_query`) para retornar uma lista de IDs ordenados de acordo com a consulta utilizando o modelo de ranking e indice que são os atributos `index` e 

In [3]:
!python -m query.tests.processing ProcessingTest.test_get_docs_term

Pesos dos documento da consulta 'crocodilo': {}

Pesos dos documento da consulta 'vocês': {2: 0.40378886090583005, 3: 0.12190858668225728}

Pesos dos documento da consulta 'Vocês estejam': {2: 0.40378886090583005, 3: 1.0168945556805116}

Pesos dos documento da consulta 'vocês vocês crocodilo': {2: 0.8075777218116601, 3: 0.24381717336451456}



.
----------------------------------------------------------------------
Ran 1 test in 0.101s

OK


**Atividade 10: Método estático runQuery e criação da interface de caracteres - ou uso da interface gráfica/web:** Você deverá implementar o método `runQuery` que utilizando o indice, valores precomputados e o dicionario de indices relevantes irá fazer:

- Instanciar um objeto de uma subclasse de RankingModel, de acordo com o que foi solicitado pelo usuário
- Preprocessar os termos da consulta
- Obter no indice as ocorrencias de cada termo do indice
- Utilize o método get_docs_term para obter a lista de documentos que responde esta consulta
- Caso seja uma consulta que possua documentos relevantes assinalados, fazer a avaliação da precisão e revocação dos top 10, 20 e 50
- Imprimir as top 10 respostas. 

Caso  opte por fazer uma interface de web, você não precisará de fazer este método em especifico  - nem o `main`, explicado a seguir - , mas, deverá possibilitar o usuário entre com uma consulta e retorne as top 10 respostas além de imprimir a precisão e revocação dos top 10, 20 e 50 das consultas que possuem documentos relevantes assinalados. 
            
Caso opte por implementar uma interface de carcteres você deverá terminar de implementar o método `main` para solicitar ao usuário a consulta e execute-a abaixo. Caso deseje, ao inves disso, você pode também fazer uma interface gráfica. Para testar este método, você deverá usar o indice da Wikipedia, assim, ele deve ser lido no início do programa. Você tem a liberdade de alterar este método como bem entender, mas lembre-se que os valores precomputados devem ser executado uma vez só durante a execução do programa antes da solicitação das consultas para que a consulta não fique lenta. Além disso, você deve ler o arquivo do índice também apenas uma vez (não indexe a Wikipedia novamente e sim leia o arquivo do indice gravado em memória). Caso deseje, você pode modificar o IndexPreComputedVals para armazenar mais elementos precomputados que facilitariam a consulta (ou a exibição da mesma). 


Para melhorar a apresentação, o arquivo `titlePerDoc.dat` apresenta o título do artigo por id do mesmo. Além disso, você deverá fazer uma análise e acordo com o [guia de escrita do relatório](https://docs.google.com/document/d/1spwD-rzJi3xHV8p5cIAmjSHyVEzytChuBTnEDwBaDNQ/edit#heading=h.bsvivts5y2ld) considerando as [tarefas do Trabalho Prático 3](https://docs.google.com/document/d/1spwD-rzJi3xHV8p5cIAmjSHyVEzytChuBTnEDwBaDNQ/edit#heading=h.fh1qug6oeoe7).


Caso opte por fazer uma interface web, deve estar claro, neste Jupyter, as instruções para que seja possível executa-la, inclusive, as suas dependencias. Para melhoria de organização, a parte de interface grafica ou web deve ser feita em outros arquivos.

In [1]:
from query.processing import QueryRunner

QueryRunner.main()

Indice lido com sucesso
Precomputando valores atraves do indice...
Precomputou valores done in 16.014716
Cleaner instanciado com sucesso
Fazendo query de 'belo horizonte'...
Query Creation done in 0.0
anwered with 1947 docs done in 0.014025
Precisao @5: 0.0
Recall @5: 0.0
Precisao @10: 0.0
Recall @10: 0.0
Precisao @20: 0.0
Recall @20: 0.0
Precisao @50: 0.0
Recall @50: 0.0
Top 10:
91853: Camargos
93107: Rio Belo
91856: Bairrocamargos
32118: Capim Branco
32436: Mateus Leme
107302: Hélio Garcia
105795: Aarão Reis
32682: São Joaquim de Bicas
32269: Florestal
9931: Alto Horizonte
Bottom 10:
129883: John Milton
103158: Identificação por radiofrequência
97042: Correspondência de Fradique Mendes
57536: Anura
67161: Fídias
12060: Língua turca
56400: Ivã IV da Rússia
49654: Fiódor Dostoiévski
24618: Santiago de Compostela
117527: Anarquismo individualista
Fazendo query de 'Belo Horizonte'...
Query Creation done in 0.0
anwered with 1947 docs done in 0.004511
Precisao @5: 0.0
Recall @5: 0.0
Precis

KeyError: 'teste'